# Python for Data Analysis
In this notebook, we are going to continue with our exposition of scientific computing with Python, particularly looking at data analysis tools in the Scipy stack. We are going to have a brief introduction to two libraries: pandas and scikit-learn.

## Pandas
Pandas - derived from 'panel data' not the cute animal :( - is a library for data analysis and manipulation. If you have used R before, you'd find some of the Pandas constructs very familiar. The main construct in Pandas is the dataframe object, which you can think of as a table where the rows are the observations and the columns are the variables/features.

## Scikit-learn
Scikit-learn is a machine learning library in Python. It implements some of the most popular machine learning algorithms as well as useful utilities for preprocessing data and validating models.

## Tutorial
This notebook is written as a tutorial on how to do a simple data analysis excercise in Python. We will be using the Boston Housing dataset which contains descriptions of houses in Boston and their corresponding prices. The task is to predict the prices from the descriptions. We start off by loading Numpy and Pandas.

In [ ]:
import numpy as np
import pandas as pd

Then we load the Boston Housing dataset. This dataset is available in scikit-learn. 

Note: if scikit-learn is not updated, the dataset will not load. This is due to Anaconda (the software we use to simplify Python library management) updated at regular interval that don't necessarily coincide with package updates. Should this happen, go to your **terminal** and type `pip install --ignore-installed --upgrade sklearn`. This will force anaconda to update the library and dependencies stated. Then **restart** your Jupyter Notebook and run the cells again.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

Let's check the type of the object that we loaded.

In [ ]:
type(boston) # Dictionary like object

This object is similar to a Python dictionary (the reason I know this is because I have looked at the docs and played around a bit!). We can look at its keys.

In [ ]:
print(boston.keys())

Let's check the types of some of the values.

In [ ]:
type(boston['data'])

In [ ]:
type(boston['target'])

Since both are Numpy arrays, we can use the stuff that we learned from the last tutorial to explore more. For instance, we can look at the shape.

In [ ]:
boston['data'].shape # 506 observations with 13 features each

Let's explore more by printing the rest of the values.

In [ ]:
print(boston['feature_names']) # Names for features

In [ ]:
print(boston['DESCR']) # Description of dataset

Now that we know what we are working with, we can store the dataset in a Pandas dataframe for further exploration and manipulation.

In [ ]:
boston_df = pd.DataFrame(boston['data'])
boston_df.head()

The `head` method in a Pandas dataframe displays the first five rows of the dataframe by default. It is a good way to take a quick look at the data. As we can see, we have 13 features, all taking numerical values. To make things more clear, we can label the columns with the feature names, to know which column corresponds to which feature.

In [ ]:
boston_df.columns = boston['feature_names']
boston_df.head()

Much better! Sadly, we are still missing one thing - the prices of the houses (target values)! We can add them to the dataframe by creating a new column and assigning it the corresponding values.

In [ ]:
boston_df['PRICE'] = boston['target']
boston_df.head()

Now that we have the complete dataframe, we can explore further. One easy way to do so is to look at summary statistics. We can easily do this with Pandas.

In [ ]:
boston_df.describe()

Here, we get the count of non-missing values, the column mean, column standard deviation, the minimum, the maximum and the 25th, 50th and 75th percentiles. These give a good picture of the statistics of the data we have. One thing to notice is that each column seems to have a different mean and standard deviation. This can cause problems when fitting a statistical/machine learning model (not always!). A good practice is to standardise the data before fitting it to a model. We will learn how to do that in Pandas. But first, let's split our data into a training and a test set. We will do a 70/30 split.

In [ ]:
train = boston_df.sample(frac=0.7) # Select a random sample of rows amounting to 70% of observations
test = boston_df.drop(train.index) # Select the remaining rows as test

In [ ]:
# Sanity check
print(train.shape)
print(test.shape)

Now that we have the split, let's standardise. To standardise a variable, we subtract it's mean and divide by its standard deviation:
$$
z = \frac{x - \mu}{\sigma}
$$
One important thing to note here is that we calculate the mean and the standard deviation on the training set only and then apply on the test set.

In [ ]:
means = train.mean(axis=0)
stds = train.std(axis=0)

In [ ]:
standard_train = (train - means) / stds
standard_train.head()

In [ ]:
standard_test = (test - means) / stds
standard_test.head()

## Task 1
I made a methodological mistake in the standardisation process. Can you spot it? Hint: look at the variable descriptions.

## Regression
Now let's fit a linear regression to the above data. We want to predict the price of the houses based on the other variables. Hence, we split the training dataframe to features and targets (features/labels, inputs/outputs, independent/dependent variables, etc...).

In [ ]:
Y_train = standard_train['PRICE']
X_train = standard_train.drop('PRICE', axis=1)

We use `scikit-learn` to fit a linear regression model rather than implementing our own as in the previous tutorial.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() # Create linear regression model object
model.fit(X_train, Y_train)

And voila! Just like that, the model is fit!!! Now we can use it to predict on new values (test values).

In [ ]:
Y_test = standard_test['PRICE']
X_test = standard_test.drop('PRICE', axis=1)
Y_predicted = model.predict(X_test)

We can see how well we did by plotting the true values against the predicted values.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.scatter(Y_test, Y_predicted)
xx = np.arange(-2, 3, 0.01)
plt.plot(xx, xx, c='red')
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices: $\hat{Y}_i$")
plt.title("Prices vs Predicted prices: $Y_i$ vs $\hat{Y}_i$") #Example of using LaTeX in plots

Not too shabby given that we have made a mistake in the preprocessing! To have a clearer idea on how well we did and to be able to compare to other model, we can calculate the mean sqaured error.

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(Y_test, Y_predicted)
print(mse)

Now we have a numerical value of our performance to compare to other models.

## Capstone project
Try to improve on the above results, either by having better data preprocessing or by using other models from the `scikit-learn` library (Look at the regression section [here](http://scikit-learn.org/stable/)). Note, if you choose not to standardise, you can do so for all the variables except the *price*, since the mean squared error comparisons won't be valid otherwise. I.e. the MSE from above will not have the same scale as the MSE from a non-standardised model, so they cannot be compared. Maybe try alternative comparison methods.